In [1]:
from cleanlab.filter import find_label_issues
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR = "../data"
SEED = 42
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.3, random_state=SEED)

In [3]:
ordered_label_issues = find_label_issues(
    labels=dataset_train['target'], #데이터셋 라벨
    pred_probs=train_pred_probs, #정답 예측 확률
    return_indices_ranked_by='self_confidence',
)

NameError: name 'train_pred_probs' is not defined

train_pred_prob 변수에 모델 학습을 시킨 뒤 정답 예측 확률을 넣어 줘야 함.

KoBERT 사용해서 구하는 걸 진행

정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
K찰.국DLwo 로L3한N% 회장 2 T0&}송=
m 김정) 자주통일 새,?r열1나가야1보
갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
pI美대선I앞두고 R2fr단 발] $비해 감시 강화
美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다
프로야구~롯TKIAs광주 경기 y천취소
아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...
朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합
듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감


In [22]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F

# 라벨 종류 개수
num_classes = data['target'].nunique()
print(f"라벨 개수 (클래스 개수): {num_classes}")

# KoBERT 토크나이저와 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=num_classes)  # num_classes는 분류할 클래스 개수

from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F

# KoBERT 토크나이저와 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=num_classes) 

# 텍스트 데이터
data_train_text = data['text'].tolist()
for i in range(10):
    print(data_train_text[i])
texts = data_train_text

# 텍스트를 모델 입력 형식으로 변환
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 모델을 평가 모드로 전환
model.eval()

# 입력 데이터를 모델에 넣어 예측
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # 출력 logits

# 예측 확률 계산
pred_probs = F.softmax(logits, dim=-1).numpy()  # (샘플 개수, 클래스 개수) 형태의 확률 배열

라벨 개수 (클래스 개수): 7


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initi

정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
K찰.국DLwo 로L3한N% 회장 2 T0&}송=
m 김정) 자주통일 새,?r열1나가야1보
갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
pI美대선I앞두고 R2fr단 발] $비해 감시 강화
美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다
프로야구~롯TKIAs광주 경기 y천취소
아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...
朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합
듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감


In [23]:
print(pred_probs)

[[0.11853136 0.16985339 0.16888973 ... 0.19313987 0.14833523 0.13044538]
 [0.11721033 0.1400339  0.1829731  ... 0.20782667 0.1174516  0.1467863 ]
 [0.16895892 0.09724345 0.1564352  ... 0.16497813 0.09021094 0.1592843 ]
 ...
 [0.14718942 0.10916314 0.18554823 ... 0.18492751 0.09553265 0.15048097]
 [0.11661261 0.1444566  0.18438815 ... 0.191192   0.12884188 0.14235795]
 [0.12152495 0.16710746 0.1703118  ... 0.2054879  0.14261377 0.11969196]]


확률 분포를 구했으니, 기존 코드에 그대로 대입해준다.

In [27]:
print("Number of labels:", len(data['target']))
print("Number of pred_probs:", len(pred_probs))

Number of labels: 2800
Number of pred_probs: 2800


In [28]:
train_pred_probs = pred_probs

ordered_label_issues = find_label_issues(
    labels=data['target'],  # 데이터셋의 실제 라벨
    pred_probs=pred_probs,  # KoBERT로 얻은 예측 확률
    return_indices_ranked_by='self_confidence',
)

head_issues = ordered_label_issues[:3]
for issue in head_issues:
    print('input text:', data.iloc[issue]['text'])
    print('label:', data.iloc[issue]['target'])
    print("------------------")

input text: 아폴로13호 선장 LG V30로 47년만에 달 착륙 꿈 이뤘다
label: 3
------------------
input text: 5산 초Q학생 3명-9가RE진…김:+Th접b8P치원q'-#확(
label: 3
------------------
input text: SKT 미래 고객 잡는다
label: 3
------------------


In [30]:
from cleanlab.dataset import health_summary
class_names=[0,1,2,3,4,5,6]
health_summary(data['target'], train_pred_probs, class_names=class_names)

----------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary          |
|   for your dataset with 2,800 examples and 7 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.        |
----------------------------------------------------------

Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,5,5,418,1,0.997613,0.500000,0.002387
1,6,6,394,3,0.997468,0.750000,0.002532
2,1,1,407,20,0.992683,0.869565,0.007317
3,0,0,393,20,0.989924,0.833333,0.010076
4,3,3,380,24,0.987013,0.827586,0.012987
5,2,2,352,166,0.907216,0.821782,0.092784
6,4,4,35,2145,0.086207,0.852544,0.913793



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,4,5,4,5,377,0.134643
1,1,4,1,4,369,0.131786
2,2,4,2,4,366,0.130714
3,4,6,4,6,360,0.128571
4,0,4,0,4,360,0.128571
5,3,4,3,4,348,0.124286
6,0,2,0,2,34,0.012143
7,1,2,1,2,32,0.011429
8,2,3,2,3,30,0.010714
9,2,6,2,6,29,0.010357



 * Overall, about 85% (2,381 of the 2,800) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 0.15.

Generated with <3 from Cleanlab.



{'overall_label_health_score': 0.1496428571428572,
 'joint': array([[0.00142857, 0.00142857, 0.01      , 0.00107143, 0.1275    ,
         0.        , 0.00035714],
        [0.00107143, 0.00107143, 0.01071429, 0.00178571, 0.13142857,
         0.00035714, 0.        ],
        [0.00214286, 0.00071429, 0.01285714, 0.00107143, 0.12178571,
         0.        , 0.        ],
        [0.00178571, 0.00178571, 0.00964286, 0.00178571, 0.1225    ,
         0.        , 0.        ],
        [0.00107143, 0.00035714, 0.00892857, 0.00178571, 0.1325    ,
         0.        , 0.00035714],
        [0.00071429, 0.00178571, 0.00964286, 0.00214286, 0.13464286,
         0.00035714, 0.00035714],
        [0.00035714, 0.00107143, 0.01035714, 0.00071429, 0.12821429,
         0.        , 0.00035714]]),
 'classes_by_label_quality':    Class Name  Class Index  Label Issues  Inverse Label Issues  Label Noise  \
 0           5            5           418                     1     0.997613   
 1           6            6  

Cleanlab 결과, KoBERT로 구한 예측 확률 분포에 전체 데이터셋을 적용해보니 85% (2381개) 의 라벨이 오류가 있다고 나온다.

현재 주어진 데이터 2800개 중에서 잘못된 라벨링이 된 데이터는 1000개 이므로, 위 결과는 잘 못 됐다.

다음 단계로, 라벨링이 정확한 1600개의 데이터를 활용하여 학습 시켜 보겠다.